In [1]:
tag2id, id2tag = {}, {}
word2id, id2word = {}, {}

for line in open('traindata.txt'):
    items = line.split('/')
    word, tag = items[0], items[1].rstrip()
    
    if word not in word2id:
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag
        
M = len(word2id)
N = len(tag2id)
print(M, N)

18978 54


In [2]:
print({key: value for key, value in word2id.items() if value < 10})
print('#' * 10)
print({key: value for key, value in tag2id.items() if value < 10})

{'Newsweek': 0, ',': 1, 'trying': 2, 'to': 3, 'keep': 4, 'pace': 5, 'with': 6, 'rival': 7, 'Time': 8, 'magazine': 9}
##########
{'NNP': 0, ',': 1, 'VBG': 2, 'TO': 3, 'VB': 4, 'NN': 5, 'IN': 6, 'JJ': 7, 'VBD': 8, 'NNS': 9}


In [3]:
import numpy as np
pi = np.zeros(N)
A = np.zeros((N, M))
B = np.zeros((N, N))

In [4]:
prev_tag = ''
for line in open('traindata.txt'):
    items = line.split('/')
    word_id, tag_id = word2id[items[0]], tag2id[items[1].rstrip()]
    if prev_tag == '':
        pi[tag_id] += 1
        A[tag_id][word_id] += 1
    else:
        A[tag_id][word_id] += 1
        B[tag2id[prev_tag]][tag_id] += 1
    if items[0] == '.':
        prev_tag = ''
    else:
        prev_tag = items[1].rstrip()

pi /= sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])

In [5]:
print(pi)
print('#' * 10)
print(A)
print('#' * 10)
print(B)

[1.81324111e-01 0.00000000e+00 1.00049407e-02 3.33498024e-03
 3.95256917e-03 3.68083004e-02 1.11660079e-01 3.66847826e-02
 6.17588933e-04 3.81669960e-02 8.76976285e-03 5.18774704e-02
 6.02766798e-02 2.47035573e-04 2.17267787e-01 0.00000000e+00
 1.48221344e-03 6.05237154e-03 8.64624506e-04 2.47035573e-04
 0.00000000e+00 4.73073123e-02 0.00000000e+00 7.16403162e-03
 1.72924901e-03 2.09980237e-03 7.53458498e-02 6.36116601e-02
 2.59387352e-03 1.85276680e-03 5.92885375e-03 1.97628458e-03
 2.84090909e-03 0.00000000e+00 0.00000000e+00 2.71739130e-03
 5.92885375e-03 5.92885375e-03 9.88142292e-04 3.70553360e-04
 1.23517787e-04 0.00000000e+00 0.00000000e+00 1.85276680e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
##########
[[5.16155673e-04 0.00000000e+00 0.00000000e+00 ... 5.16155673e-05
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.99801725e-01 0.00000000e+00 ... 0.00000000e+00

In [8]:
def log(v):
    if v == 0:
        return np.log(v + 0.0000001)
    return np.log(v)

In [11]:
def viterbi(x, pi, A, B):
    """
    x: user input string/sentence
    pi: initial probability of tags
    A: 
    """
    x = [word2id[word] for word in x.split(' ')]
    T = len(x)
    dp = np.zeros((T, N)) # dp[i][j], i is number of word, j is the number of tag
    pointer = np.array([[0 for x in range(N)] for y in range(T)])
    for j in range(N): # basecase
        dp[0][j] = log(pi[j]) + log(A[j][x[0]])
    for i in range(1, T): # loop words
        for j in range(N): # loop tags
            dp[i][j] = -float('inf')
            for k in range(N): # loop through last word's tag
                score = dp[i - 1][k] + log(B[k][j]) + log(A[j][x[i]])
                if score > dp[i][j]:
                    dp[i][k] = score
                    pointer[i][j] = k
    best_seq = [0] * T
    best_seq[T - 1] = np.argmax(dp[T - 1])
    for i in range(T - 2, -1, -1):
        best_seq[i] = pointer[i + 1][best_seq[i + 1]]
    for i in range(len(best_seq)):
        print(id2tag[i])

In [12]:
x = "Social Security number , passport number and details about the services provided for the payment"
viterbi(x, pi, A, B)

NNP
,
VBG
TO
VB
NN
IN
JJ
VBD
NNS
CD
CC
PRP
MD
DT
